# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.2/884.2 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72

In [2]:
import duckdb
import leafmap
import pandas as pd

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [3]:
#新建空的db
con = duckdb.connect()

In [4]:
con.install_extension('httpfs')
con.load_extension('httpfs')

In [5]:
con.install_extension('spatial')
con.load_extension('spatial')

In [6]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 77.9MB/s]


Extracting files...


'/content/nyc_data.zip'

In [ ]:
#查看可以读取的格式
con.sql('SELECT * FROM ST_Drivers()')

In [7]:
#读取shp文件
con.sql("""
        CREATE TABLE IF NOT EXISTS nyc_census_blocks AS
        SELECT * FROM ST_Read('nyc_census_blocks.shp')
""")

In [8]:
con.sql("CREATE TABLE nyc_homicides AS SELECT * FROM ST_Read('nyc_homicides.shp')")

In [10]:
con.sql("CREATE TABLE nyc_streets AS SELECT * FROM ST_Read('nyc_streets.shp')")

In [9]:
con.sql("CREATE TABLE nyc_neighborhoods AS SELECT * FROM ST_Read('nyc_neighborhoods.shp')")

In [11]:
con.sql("CREATE TABLE nyc_subway_stations AS SELECT * FROM ST_Read('nyc_subway_stations.shp')")

In [12]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

<font color=#DE3163>区分：

- <font color=#DE3163>`ST_Intersects()`：**是否**相交**（部分重叠或接触）

- <font color=#DE3163>`ST_Contains(geomA， geomB）` :检查一个几何 （`geomA`） 是否**完全包含**另一个几何 （`geomB`）

- 如果你不想让“边界上的地铁站”出现在查询结果里，建议用 `ST_Contains(n.geom, s.geom)`。 如果你想查询所有“在社区内部或边界上的地铁站”，保持 `ST_Intersects()` 即可！

In [18]:
con.sql("""
SELECT *
FROM nyc_neighborhoods
WHERE name='Little Italy';
""")

┌───────────┬──────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │     NAME     │                                                                                                                                     geom                                                                                                                                     │
│  varchar  │   varchar    │                                                                                                                                   geometry                                                                                                                                   │
├───────────┼──────────────┼────────────────────────────────────────────────────────────────────────

In [21]:
con.sql("""
SELECT n.name, s.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Intersects(s.geom, n.geom)
WHERE n.name='Little Italy';
""")

┌──────────────┬───────────┬─────────┐
│     NAME     │   NAME    │ ROUTES  │
│   varchar    │  varchar  │ varchar │
├──────────────┼───────────┼─────────┤
│ Little Italy │ Spring St │ 6       │
└──────────────┴───────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [16]:
con.sql("""
SELECT DISTINCT n.name
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_DWithin(s.geom, n.geom,200)
WHERE strpos(s.routes,'6') > 0;
""")

┌────────────────────┐
│        NAME        │
│      varchar       │
├────────────────────┤
│ Parkchester        │
│ Hunts Point        │
│ South Bronx        │
│ Soundview          │
│ Yorkville          │
│ Harlem             │
│ Greenwich Village  │
│ Murray Hill        │
│ Midtown            │
│ Upper East Side    │
│ Carnegie Hill      │
│ Chinatown          │
│ Mott Haven         │
│ Union Port         │
│ East Harlem        │
│ Gramercy           │
│ East Village       │
│ Soho               │
│ Little Italy       │
│ Tribeca            │
│ Financial District │
│ Lower East Side    │
├────────────────────┤
│      22 rows       │
└────────────────────┘

<font color=#DE3163>答案没有扩展范围

In [25]:
con.sql("""
SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0
ORDER BY n.name; -- 按照首字母排序
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Chinatown          │ Manhattan │
│ East Harlem        │ Manhattan │
│ Financial District │ Manhattan │
│ Gramercy           │ Manhattan │
│ Greenwich Village  │ Manhattan │
│ Hunts Point        │ The Bronx │
│ Little Italy       │ Manhattan │
│ Midtown            │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Murray Hill        │ Manhattan │
│ Parkchester        │ The Bronx │
│ Soundview          │ The Bronx │
│ South Bronx        │ The Bronx │
│ Upper East Side    │ Manhattan │
│ Yorkville          │ Manhattan │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [36]:
con.sql("""
SELECT *
FROM nyc_neighborhoods
WHERE name ='Battery Park';
""")

┌───────────┬──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [26]:
con.sql("""
SELECT sum(c.popn_total) AS population
FROM nyc_neighborhoods as n
JOIN nyc_census_blocks as c
ON ST_Intersects(n.geom,c.geom)
WHERE n.name ='Battery Park';
""")

┌────────────┐
│ population │
│   int128   │
├────────────┤
│      17153 │
└────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


注意单位

In [27]:
con.sql("""
SELECT n.name,
sum(c.popn_total)/(ST_Area(n.geom)/1000000) AS population
FROM nyc_neighborhoods as n
JOIN nyc_census_blocks as c
ON ST_Intersects(n.geom,c.geom)
GROUP BY n.name, n.geom
ORDER BY population DESC
LIMIT 5;
""")

┌───────────────────┬───────────────────┐
│       NAME        │    population     │
│      varchar      │      double       │
├───────────────────┼───────────────────┤
│ North Sutton Area │ 68435.13283772678 │
│ East Village      │ 50404.48341332535 │
│ Chinatown         │  48825.1805506297 │
│ Carnegie Hill     │ 48543.72540441477 │
│ Upper East Side   │ 48524.48774898572 │
└───────────────────┴───────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. **The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other.**This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [1]:
%pip install duckdb duckdb-engine jupysql

In [2]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [3]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [4]:
%%sql
SELECT * FROM 'https://storage.googleapis.com/qm2/casa0025_ships.csv'

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,POINT (36.5133266666667 45.1517766666667)
1,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,POINT (36.52078 45.1464866666667)
2,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,POINT (36.521965 45.1462183333333)
3,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,POINT (36.52202 45.1450583333333)
4,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,POINT (36.5218483333333 45.1449333333333)
...,...,...,...,...,...,...,...,...,...,...,...
101323,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:16:47,45.091987,36.522157,POINT (36.5221566666667 45.0919866666667)
101324,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,0.1,2022-08-10 14:43:48,45.091643,36.522213,POINT (36.5222133333333 45.0916433333333)
101325,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,5.8,2022-08-10 15:04:28,45.100457,36.519397,POINT (36.5193966666667 45.1004566666667)
101326,217531,Zubeyde,roll on roll off with container capacity,5000.0,113.0,4.5,8.3,2022-08-23 06:06:51,45.087527,36.506987,POINT (36.5069866666667 45.0875266666667)


In [4]:
%%sql

CREATE TABLE ais AS
SELECT *,
ST_GeomFromText(geom) AS geometry
FROM 'https://storage.googleapis.com/qm2/casa0025_ships.csv';

,Success


In [5]:
%%sql
CREATE TABLE vinfo AS SELECT *,
ST_GeomFromText(geom) AS geometry
FROM 'https://storage.googleapis.com/qm2/casa0025_ships.csv';

,Success


In [6]:
%%sql

CREATE INDEX ais_index
ON ais USING RTREE(geometry);

,Success


In [7]:
%%sql
CREATE INDEX vinfo_index
ON vinfo USING RTREE(geometry);

,Success


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [ ]:
%%sql

SELECT ais.vesselid,vinfo.vesselid
FROM ais
JOIN vinfo
ON ST_DWithin(ais.geometry, vinfo.geometry,500)
WHERE ABS(EXTRACT(EPOCH FROM (ais.date - vinfo.date))) > 7200;



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
%%sql

SELECT ais.vesselid
FROM ais
JOIN vinfo
ON ST_DWithin(ais.geometry, vinfo.geometry, 500)
AND ABS(EXTRACT(EPOCH FROM (ais.date - vinfo.date))) > 7200
AND ais.vesselid <> vinfo.vesselid
WHERE ais.sog < 1 AND vinfo.sog < 1;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(651): get
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(700): __get__
  /usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_float.py(37): _validate_value
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(1229): __call__
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(743): _cross_validate
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(737): _validate
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(703): set
  /usr/local/lib/python3.11/dist-packages/traitlets/traitlets.py(729): __set__
  /usr/local/lib/python3.11/dist-packages/sql/run/resultset.py(527): _convert_to_data_frame
  /usr/local/lib/python3.11/dist-packages/sql/run/resultset.py(252): DataFrame
  /usr/local/lib/python3.11/dist-packages/sql/run/run.py(84): select_df_type
  /usr/local/lib/python3.11/dist-packages/sql/run/